In [1]:
import pandas as pd #dataframes

In [2]:
#create a dataframe for the months needed after importing the files
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/fhv_tripdata_2020-"+month+".csv" , low_memory = False), ignore_index=True)
    
        
#rename the location id to match the taxi zone location id         
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [3]:
#create a new df from selected columns
df2 = df[['LocationID', 'pickup_datetime']]

#change the datetime to date
df2['pickup_datetime'] = df2['pickup_datetime'].dt.date

#create a dataframe by merging with taxi zone
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
fhv_2020_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)
fhv_2020_boroughs = fhv_2020_boroughs.drop(['LocationID'], axis=1)
fhv_2020_boroughs

<ipython-input-3-eb88e5bb7cd0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pickup_datetime'] = df2['pickup_datetime'].dt.date


,pickup_datetime,Borough
0,2020-03-01,Unknown
1,2020-03-01,Unknown
2,2020-03-01,Unknown
3,2020-03-01,Unknown
4,2020-03-01,Unknown
...,...,...
1607349,2020-04-07,Manhattan
1607350,2020-05-05,Manhattan
1607351,2020-03-04,Queens
1607352,2020-03-05,Queens


In [4]:
#create a dataframe with only usage
fhv_2020= pd.DataFrame(df2.pickup_datetime.value_counts().reset_index().values, columns=["Date", "Fhv_2020"])
fhv_2020 = fhv_2020.sort_values(by="Date")
fhv_2020

,Date,Fhv_2020
12,2020-03-01,44800
4,2020-03-02,65168
2,2020-03-03,68632
1,2020-03-04,69476
0,2020-03-05,69880
...,...,...
16,2020-06-26,37655
38,2020-06-27,29040
62,2020-06-28,23562
17,2020-06-29,37264


In [5]:
#save the dataframe as csv file
fhv_2020.to_csv("fhv_2020.csv")

In [6]:
#create a dataframe to get the usage by borough
fhv_2020_borough = fhv_2020_boroughs.groupby("Borough").count().reset_index()
fhv_2020_borough .rename(columns={'pickup_datetime':'Fhv_2020'}, inplace=True)
fhv_2020_borough

,Borough,Fhv_2020
0,Bronx,51538
1,Brooklyn,63230
2,EWR,738
3,Manhattan,62113
4,Queens,84774
5,Staten Island,29300
6,Unknown,1315661


In [7]:
#save the dataframe as csv file
fhv_2020_borough.to_csv("fhv_2020_borough.csv")